In [ ]:
#Import stuff
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
df = pd.read_csv("./../../data/reddit_data.csv", encoding='utf-8')
df.created = pd.to_datetime(df.created)
df.title = df.title.astype(str)
df.selftext = df.selftext.astype(str)
df.dtypes

title                   object
score                    int64
id                      object
subreddit               object
url                     object
num_comments             int64
num_upvotes              int64
selftext                object
created         datetime64[ns]
dtype: object

In [ ]:
import re
#Remove URLs
df["title"] = df["title"].apply(lambda x : re.sub(r"http\S+|www\S+|https\S+", "", x))
df["selftext"] = df["selftext"].apply(lambda x : re.sub(r"http\S+|www\S+|https\S+", "", x))

In [47]:
df.head()
df.dtypes

title                   object
score                    int64
id                      object
subreddit               object
url                     object
num_comments             int64
num_upvotes              int64
selftext                object
created         datetime64[ns]
dtype: object

In [ ]:
def score_to_sentiment(score : float, threshold = 0.05):

    #Using threshold 0.05 as recommended by the VADER documentation
    if score > threshold:
        return "positive"
    elif score < -threshold:
        return "negative"
    else:
        return "neutral"
#Using the VADER sentiment analysis tool
vader = SentimentIntensityAnalyzer()
df["vader_title_sentiment"] = df["title"].apply(lambda x : vader.polarity_scores(x)["compound"])
df["vader_selftext_sentiment"] = df["selftext"].apply(lambda x : vader.polarity_scores(x)["compound"])
df["vader_sentiment"] = df["selftext"].apply(
    lambda x : score_to_sentiment(vader.polarity_scores(x)["compound"])
    )

In [53]:
df = df.rename(columns={
    'title_sentiment': 'vader_title_sentiment',
    'selftext_sentiment': 'vader_selftext_sentiment',
    'sentiment': 'vader_sentiment'
})
df.sample(1)

,title,score,id,subreddit,url,num_comments,num_upvotes,selftext,created,vader_title_sentiment,vader_selftext_sentiment,vader_sentiment
512,care much days...,1,1jfz4tg,depression_help,https://www.reddit.com/r/depression_help/comme...,1,1,"ive depressed life, slowly inch inch feel like...",2025-03-20 20:44:44,0.4939,0.9767,positive


In [54]:
df.to_csv("./../../data/vader_sentiment.csv", index=False, encoding='utf-8')